# Embedding sample notebook

The following notebook demonstartes how to embed your Power BI content in a Jupyter notebook and interact with it.
To run the demo, you will need to obtain your report id and group (workspace) id from the URL of your report.

In [1]:
from powerbiclient.authentication import InteractiveLoginAuthentication
from powerbiclient import Report, models
from io import StringIO
from ipywidgets import interact
import requests
import pandas as pd
import matplotlib

### Authenticate to Power BI


#### The following authentication methods can be used to authenticate the user and get the access token:

- [Device flow authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_by_device_flow)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import DeviceCodeLoginAuthentication
  device_auth = DeviceCodeLoginAuthentication()
    ```
- [Interactive authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_interactive)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import InteractiveLoginAuthentication
  interactive_auth = InteractiveLoginAuthentication()    
    ```

Note: Either use auth object created or access token acquired above while creating the report instance


#### Run the cell below to obtain the auth object using the Device Flow Authentication method for the demo notebook

In [2]:
# Auth object for demo
auth = InteractiveLoginAuthentication()

A local browser window will open for interactive sign in.

Interactive authentication successfully completed.
You are now logged in.

The result should be passed only to trusted code in your notebook.


### Create an instance of Power BI Report

- Below snippets are the values for optional view_mode parameter
``` py 
view_mode=models.EmbedMode.VIEW.value
view_mode=models.EmbedMode.EDIT.value
view_mode=models.EmbedMode.CREATE.value
``` 
Default value for the parameter is 
``` py 
view_mode=models.EmbedMode.VIEW.value 
```

- Below snippets are the values for the [permissions](https://docs.microsoft.com/en-us/javascript/api/overview/powerbi/configure-report-settings#open-in-edit-mode) parameter for embedding the report in EDIT mode
``` py 
permissions=models.Permissions.READWRITE.value
permissions=models.Permissions.COPY.value
permissions=models.Permissions.ALL.value
``` 

#### Render report in Create mode
To create a new report in embed mode, the following additional parameters need to be passed
``` py
dataset_id='a87ccf23-f2ce-4fde-a095-90e45d850503'
view_mode=models.EmbedMode.CREATE.value
```

#### Replace the group_id and report_id parameters with your own report ID and group ID and run the cell below to create a Power BI report instance:

In [3]:
dataset_id='a87ccf23-f2ce-4fde-a095-90e45d850503'
view_mode=models.EmbedMode.CREATE.value
group_id = "51ada905-6b70-4ab8-8416-75d4c33fca67"
report_id = "f92cde0f-e99d-4fa6-a401-e3610798dc3f"
report = Report(group_id=group_id, report_id=report_id, auth=auth)

Report is loaded
Report is rendered
Report is rendered


### Power BI event handlers

#### Set report loaded event handler

In [10]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

#### Set report rendered event handler

In [11]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

#### Load the report in the output cell

In [12]:
# Render report
report

Report(container_height=600.0, container_width=900.0)

#### Change dimensions of embedded report

In [13]:
# Change dimensions of embedded report
report.set_size(600, 900)

### Export data from a visual of the embedded report

#### Get the active page

In [14]:
def get_active_page(self):
    
    # Get list of pages
    pages = self.get_pages()
    active_page = {}
    for page in pages:
        
        # Get active page
        if page['isActive'] == True:
            active_page = page
            break
    return active_page

#### Get the visual whose data is to be exported

Make sure that the active page on the report you're embedding has a visual of type 'clustered column chart', or change to a different visual type below.

In [22]:
active_page = get_active_page(report)
active_page_name = active_page['name']

# Get list of visuals on active page
visuals = report.visuals_on_page(active_page_name)

# Get visual whose type is 'clusteredColumnChart'
visual = next(filter(lambda visual: visual['type'] == 'lineChart', visuals))
visual_name = visual['name']

#### Export summarized data from a visual of the embedded report

In [23]:
summarized_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 20)

print(summarized_exported_data)

created,Sum of id
2018-01-08 19:18:38,1372479
2019-10-30 21:24:14,2987100
2020-04-13 22:46:09,3253635
2020-06-25 22:20:54,3350139
2020-06-30 15:18:36,3353925
2020-08-24 18:36:20,3407118
2020-10-06 16:28:29,3448983
2020-10-21 20:49:23,3464679
2020-10-21 22:33:03,3464790
2020-10-21 23:30:30,3464826
2021-01-12 23:15:32,3523338
2021-01-13 15:23:43,3524937
2021-01-13 15:32:30,3524973
2021-01-13 15:38:00,3524991
2021-01-13 16:05:56,3525072
2021-01-18 11:37:26,3527760
2021-01-19 15:06:28,3529332
2021-01-22 16:08:26,3533553
2021-01-27 14:14:55,3536544
2021-05-20 19:55:36,3636462



#### Export underlying data from a visual of the embedded report

In [24]:
underlying_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 5, export_data_type = models.ExportDataType.UNDERLYING.value)

print(underlying_exported_data)

created,Sum of id,description,summary,issue_key
2018-01-08 19:18:38,457493,"Epic Purpose: tech debt and adhoc analysis tickets

Also See

[https://rackspace.atlassian.net/browse/DATA-29305|https://rackspace.atlassian.net/browse/DATA-29305|smart-link]

[https://rackspace.atlassian.net/browse/DATA-29304|https://rackspace.atlassian.net/browse/DATA-29304|smart-link]



-Originally created-

-This charter will cover all Global Data efforts capturing system enhancements covering productivity, overlay, focus and automation for the Enterprise.   Enhancements are classified as  a noteworthy improvement to a product as part of a new version of it that enables and provides new funcationality to the business. Any enhancements or changes to CORP DMART requested from the enterprise.-

-Systems In Scope - Corporate DMART.-",Engineering AdHoc,DATA-137
2018-01-08 19:18:38,457493,"Epic Purpose: tech debt and adhoc analysis tickets

Also See

[https://rackspace.atlassian.net/browse/DATA-29305|https://rac

### Convert exported visual data into CSV and load into pandas dataframe

In [25]:
data = StringIO(summarized_exported_data)

df = pd.read_csv(data, sep=",")
df.head()

,created,Sum of id
0,2018-01-08 19:18:38,1372479
1,2019-10-30 21:24:14,2987100
2,2020-04-13 22:46:09,3253635
3,2020-06-25 22:20:54,3350139
4,2020-06-30 15:18:36,3353925


### Visualize the data

Adjust the plot parameters below to fit your report

In [ ]:
df.plot.pie(y="Total Units", labels=df.loc[:,'isVanArsdel'].values.tolist(), figsize=(10,15))

### Filter the report by region using iPyWidget

Adjust the filter below to fit your report

In [ ]:
def filter_report(Region):
    region_filter = {
        '$schema': "http://powerbi.com/product/schema#basic",
        'target': {
            'table': "Geo",
            'column': "Region"
        },
        'operator': "In",
        'values': [Region]
    }
    report.remove_filters()
    report.update_filters([region_filter])

In [ ]:
report

In [ ]:
interact(filter_report, Region=['Central','East', 'West']);

### Get filters applied on the report

In [ ]:
report.get_filters()

### Get report bookmarks

In [ ]:
# Get list of bookmarks
bookmarks_list = report.get_bookmarks()

# Convert the list into a dataframe
bookmarks_dataframe = pd.DataFrame(bookmarks_list)

# Rename columns
bookmarks_dataframe.rename(columns={'name':'Name', 'displayName':'Display Name', 'state':'State'}, inplace=True)
bookmarks_dataframe

### Apply report bookmark

In [ ]:
def apply_bookmark(BookmarkDisplayName):
    
    # Get bookmark name corresponding to the display name provided by the user
    bookmark_name = bookmarks_dataframe[bookmarks_dataframe['Display Name'] == BookmarkDisplayName].Name.values[0]
    report.set_bookmark(bookmark_name)

### Apply bookmark using ipywidget

In [ ]:
report

In [ ]:
interact(apply_bookmark, BookmarkDisplayName=bookmarks_dataframe['Display Name'])